# 📓 03_stableford_simulation.ipynb
**Purpose:** Corrected Monte Carlo simulation using Stableford scoring based on per-hole par values.

In [ ]:
import pickle
import random
import numpy as np
from collections import defaultdict
from golf_classes import Player, PlayerRoundInfo, Tournament, Round, Team

In [ ]:
# --- Load Data ---

def load_data(filename='golf_data.pkl'):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data['players'], data['tournaments']

PICKLE_FILE = 'golf_data.pkl'  
players, tournaments = load_data(PICKLE_FILE)
print(f"✅ Loaded {len(players)} players and {len(tournaments)} tournaments.")

In [ ]:
# --- Course Setup ---
hole_handicap_ratings = [5, 13, 17, 3, 11, 9, 1, 15, 7, 10, 6, 18, 14, 2, 16, 4, 12, 8]
hole_pars = [4, 4, 3, 4, 3, 4, 4, 5, 4, 4, 4, 3, 5, 4, 3, 4, 5, 4]

In [ ]:
# --- Compute strokes received per hole ---
def strokes_received_per_hole(player_handicap):
    strokes = [0] * 18
    for i in range(18):
        hcap = hole_handicap_ratings[i]
        if player_handicap >= hcap:
            strokes[i] += 1
        if player_handicap > 18 and player_handicap >= hcap + 18:
            strokes[i] += 1
    return strokes

In [ ]:
# --- Convert net score to Stableford points ---
def stableford_points(net_score):
    if net_score >= 2:
        return -2  # Double bogey or worse
    elif net_score == 1:
        return 0   # Bogey
    elif net_score == 0:
        return 1   # Par
    elif net_score == -1:
        return 3   # Birdie
    elif net_score == -2:
        return 5   # Eagle
    else:
        return 7   # Double eagle or better


In [ ]:
# --- Calculate Stableford points for a round ---
def calculate_stableford_round(player_round):
    if not player_round.hole_scores or len(player_round.hole_scores) != 18:
        return None

    gross_scores = player_round.hole_scores
    handicap = player_round.handicap
    strokes = strokes_received_per_hole(handicap)

    total_points = 0
    for i in range(18):
        try:
            gross_score = float(gross_scores[i])
        except (ValueError, TypeError):
            return None

        net = gross_score - strokes[i]
        net_relative_to_par = round(net - hole_pars[i])
        stableford = stableford_points(net_relative_to_par)

        total_points += stableford

    return total_points

In [ ]:
# --- Sample Stableford score for a player ---
def sample_stableford_score(player):
    if not player.rounds:
        return None
    round_info = random.choice(player.rounds)
    return calculate_stableford_round(round_info)

In [ ]:
# --- Simulate team Stableford score ---
def simulate_team_stableford_score(team):
    player_scores = []
    for player in team.members:
        score = sample_stableford_score(player)
        if score is not None:
            player_scores.append(score)

    if len(player_scores) < 4:
        return float('-inf')  # Invalid team

    return sum(sorted(player_scores, reverse=True)[:4])

In [ ]:
# --- Simulate Stableford tournament ---
def simulate_stableford_tournament(teams, num_simulations=10000):
    team_scores = defaultdict(list)

    for _ in range(num_simulations):
        for team in teams:
            score = simulate_team_stableford_score(team)
            team_scores[team.name].append(score)

    team_averages = {name: np.mean(scores) for name, scores in team_scores.items()}
    return team_averages

In [ ]:
# --- Example Team Class and Simulation ---
class Team:
    def __init__(self, name, members):
        self.name = name
        self.members = members

# Random example teams
example_teams = [
    Team("Team A", random.sample(list(players.values()), 6)),
    Team("Team B", random.sample(list(players.values()), 6))
]

# Run simulation
results = simulate_stableford_tournament(example_teams, num_simulations=10000)

# Print results
print("\n🏆 Team Stableford Averages:")
for name, avg in sorted(results.items(), key=lambda x: -x[1]):
    print(f"{name}: {avg:.2f} points")